In [2]:
import numpy as np
import pandas as pd

In [9]:
class Neural_Network:
    def __init__(self, net):
        self.layers = len(net)
        self.net = net
        self.biases = [np.random.randn(j,1) for j in net[1:]]
        self.weights = [np.random.randn(j,k) for k,j in zip(net[:-1], net[1:])]
    def sigmoid(self, z):
        a = 1/(1+np.exp(-z))
        return a
    def feedforward(self, a):
        for w,b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w,a) + b)
        return a
    def SDG(self, data, epochs, mini_batch_size, testing_data=False):
        if testing_data:
            return self.evaluation()
        data_length = len(data)
        for x in range(epochs):
            np.random.shuffle(data)
            for batch in batched(data, mini_batch_size):
                self.mini_batch_update(batch)
                
            
    def mini_batch_update(self, batch):
        db = [np.zeros(i.shape()) for i in self.biases]
        dw = [np.zeros(i.shape()) for i in self.weights]
        for i in batch:
            back_propagation(i)
            
    def back_propagation(self, x, label):
        #input
        #feedforward
        a = feedforward(x)
        #output error
        #backpropagate the error
        #output
        pass
    def evaluation(self):
        pass

In [6]:
x = [1,2,3,4,5,6]
for i in range(x, 2):
    print(i)

TypeError: 'list' object cannot be interpreted as an integer